In [106]:
from alpha_vantage.fundamentaldata import FundamentalData
import pandas as pd
import time
import sqlite3

In [107]:
apikey = '4YLKM5SFV5RXMQCG'
fd = FundamentalData(apikey, output_format = 'pandas')

symbol = 'ACAD'

data = fd.get_balance_sheet_quarterly(symbol)
data1 = fd.get_income_statement_quarterly(symbol)
data = data[0].T
data1 = data1[0].T

In [108]:
result = pd.concat([data, data1])
result = result.T.reset_index()
new_result = result.loc[:,~result.columns.duplicated(keep='first')]
new_result = new_result.drop(new_result.columns[[0,2]], axis=1)

new_result['netIncome'] = result['netIncome']
new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float)

Equity = (new_result['totalAssets'] - new_result['totalLiabilities'])
NetProfit = (new_result['grossProfit'] - new_result['operatingExpenses'])
ROA_value = (new_result['netIncome'] / new_result['totalAssets']) * 100
ROE_value = (new_result['netIncome'] / new_result['totalShareholderEquity']) * 100

new_result['Equity'] = Equity
new_result['NetProfit'] = NetProfit
new_result['ROA'] = ROA_value
new_result['ROE'] = ROE_value
new_result['Symbol'] = symbol

C:\Users\Admin\AppData\Local\Temp\ipykernel_15184\3633238364.py:7: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(lambda x: pd.to_numeric(x, errors='coerce')).astype(float)


In [109]:
result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
                                          31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]]
                                          , axis=1)
result_after = result_after.round(3)
result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
                             "totalRevenue": "Revenue"})
result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'Symbol']]
result_after['Period'] = pd.to_datetime(result_after['Period'])
result_after = result_after[~(result_after['Period'].isna())]
result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
result_after['Period'] = result_after['Period'].dt.quarter.astype(str) +'Q'+ result_after['Period'].dt.year.astype(str)

result_after

,Asset,TotalDebt,Equity,Revenue,NetProfit,ROA,ROE,Period,Symbol
0,351040000.0,46871000.0,304169000.0,48868000.0,-53865000.0,-15.467,-17.851,1Q2018,ACAD
1,314960000.0,50240000.0,264720000.0,57063000.0,-62963000.0,-20.087,-23.899,2Q2018,ACAD
2,266456000.0,43016000.0,223440000.0,58305000.0,-60135000.0,-23.320,-27.810,3Q2018,ACAD
3,540202000.0,61123000.0,479079000.0,59571000.0,-66136000.0,-12.124,-13.670,4Q2018,ACAD
4,501105000.0,82749000.0,418356000.0,62959000.0,-86374000.0,-17.023,-20.390,1Q2019,ACAD
5,460883000.0,71969000.0,388914000.0,83205000.0,-55454000.0,-11.921,-14.127,2Q2019,ACAD
6,765586000.0,76297000.0,689289000.0,94586000.0,-43548000.0,-5.483,-6.090,3Q2019,ACAD
7,783183000.0,84048000.0,699135000.0,98326000.0,-54457000.0,-6.772,-7.586,4Q2019,ACAD
8,747450000.0,109068000.0,638382000.0,90068000.0,-87710000.0,-11.776,-13.788,1Q2020,ACAD
9,759649000.0,96636000.0,663013000.0,110103000.0,-41835000.0,-5.547,-6.356,2Q2020,ACAD


In [110]:
result_after.dtypes

Asset        float64
TotalDebt    float64
Equity       float64
Revenue      float64
NetProfit    float64
ROA          float64
ROE          float64
Period        object
Symbol        object
dtype: object

In [111]:
# def alphaVantage_data(symbol):
#     apikey = '4YLKM5SFV5RXMQCG'
#     fd = FundamentalData(apikey, output_format = 'pandas')
#     balance_sheet = fd.get_balance_sheet_quarterly(symbol)
#     income_state = fd.get_income_statement_quarterly(symbol)
#     balance_sheet = balance_sheet[0].T
#     income_state = income_state[0].T
#     return (balance_sheet, income_state)

# def calInfo(data):
#     Equity = (data['totalAssets'] - data['totalLiabilities'])
#     NetProfit = (data['grossProfit'] - data['operatingExpenses'])
#     ROA_value = (data['netIncome'] / data['totalAssets']) * 100
#     ROE_value = (data['netIncome'] / data['totalShareholderEquity']) * 100
#     data['Equity'] = Equity
#     data['NetProfit'] = NetProfit
#     data['ROA'] = ROA_value
#     data['ROE'] = ROE_value
#     return data

# def arrangeAlphaVantageDetail(symbol):
#     data_balance, data_income = alphaVantage_data(symbol)
#     result = pd.concat([data_balance, data_income])
#     result = result.T.reset_index()
#     new_result = result.loc[:,~result.columns.duplicated(keep='first')]
#     new_result = new_result.drop(new_result.columns[[0,2]], axis=1)
#     new_result['netIncome'] = result['netIncome']
#     new_result.iloc[:, 1:] = new_result.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
#     new_result = calInfo(new_result)
#     new_result['Symbol'] = symbol
#     result_after = new_result.drop(new_result.columns[[2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,19,20,21,22,23,24,25,26,27,28,29,30,
#                                           31,32,33,34,35,36,37,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60]], axis=1)
#     result_after = result_after.round(3)
#     result_after = result_after.rename(columns={"fiscalDateEnding": "Period", "totalAssets": "Asset", "totalLiabilities": "TotalDebt",
#                                 "totalRevenue": "Revenue"})
#     result_after = result_after[['Asset', 'TotalDebt', 'Equity', 'Revenue', 'NetProfit', 'ROA', 'ROE', 'Period', 'Symbol']]
#     result_after['Period'] = pd.to_datetime(result_after['Period'])
#     result_after = result_after.sort_values(by=['Period'], ascending=True).reset_index(drop=True)
#     result_after['Period'] = result_after['Period'].dt.quarter.astype(str) + 'Q' + result_after['Period'].dt.year.astype(str)
#     return result_after

In [112]:
# result = arrangeAlphaVantageDetail('META')
# result

In [113]:
# def AlphaVantageDetail():
#     values = allSymbolnasdaq()[:50]
#     details_alpha = []
#     count = 0
#     for name in values:
#         if (count == 2):
#             time.sleep(65)
#             df = arrangeAlphaVantageDetail(name)
#             df_old = details_alpha[0]
#             df_new = pd.concat([df_old, df])
#             details_alpha[0] = df_new
#             count = 0
#         elif len(details_alpha) == 0:
#             df1 = arrangeAlphaVantageDetail(name)
#             details_alpha.append(df1)
#         else:
#             df2 = arrangeAlphaVantageDetail(name)
#             df_old = details_alpha[0]
#             df_new = pd.concat([df_old, df2])
#             details_alpha[0] = df_new
#         count+=1
#     return details_alpha[0]